# OpenAI Tools

These output parsers extract tool calls from OpenAI’s function calling API responses. This means they are only usable with models that support function calling, and specifically the latest `tools` and `tool_choice` parameters. We recommend familiarizing yourself with [function calling](/docs/modules/model_io/chat/function_calling) before reading this guide.

There are a few different variants of output parsers:

- [`JsonOutputToolsParser`](https://api.js.langchain.com/classes/langchain_output_parsers.JsonOutputToolsParser.html): Returns the arguments of the function call as JSON
- [`JsonOutputKeyToolsParser`](https://api.js.langchain.com/classes/langchain_output_parsers.JsonOutputKeyToolsParser.html): Returns the value of specific key in the function call as JSON

In [1]:
import { ChatPromptTemplate } from '@langchain/core/prompts';
import { ChatOpenAI } from '@langchain/openai';

In [3]:
const properties = {
  setup: {
    type: "string",
    description: "The setup for the joke"
  },
  punchline: {
    type: "string",
    description: "The joke's punchline"
  }
};

const tool = {
  type: "function" as const,
  function: {
    name: "joke",
    description: "Joke to tell user.",
    parameters: {
      $schema: "http://json-schema.org/draft-07/schema#",
      title: "Joke",
      type: "object",
      properties,
      required: ["setup", "punchline"]
    },
  },
}

In [4]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const llm = new ChatOpenAI();

// Use `.bind` to attach the tool to the model
const llmWithTools = llm.bind({
  tools: [tool],
  // Optionally, we can pass the tool to the `tool_choice` parameter to
  // force the model to call the tool.
  tool_choice: tool,
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are the funniest comedian, tell the user a joke about their topic."],
  ["human", "Topic: {topic}"]
])

Now we can use LCEL to pipe our prompt and LLM together.

In [11]:
const chain = prompt.pipe(llmWithTools);

In [12]:
JSON.stringify(await chain.invoke({ topic: "Large Language Models" }), null, 2);

"{\n" +
  '  "lc": 1,\n' +
  '  "type": "constructor",\n' +
  '  "id": [\n' +
  '    "langchain_core",\n' +
  '    "messages",\n' +
  '    "AIMessage'... 454 more characters

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/2413c983-f803-4bea-8ea1-adadf6a0af32/r) from the call above

## JsonOutputToolsParser

In [13]:
import { JsonOutputToolsParser } from "langchain/output_parsers";

const outputParser = new JsonOutputToolsParser();

In [14]:
const chain = prompt.pipe(llmWithTools).pipe(outputParser);

In [15]:
JSON.stringify(await chain.invoke({ topic: "Large Language Models" }), null, 2);

"[\n" +
  "  {\n" +
  '    "type": "joke",\n' +
  '    "args": {\n' +
  '      "setup": "Why did the large language model start a ban'... 84 more characters

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/1981c0f9-6059-4f4b-a29e-579aa43c571b/r) with the `JsonOutputToolsParser` output parser

## JsonOutputKeyToolsParser

This merely extracts a single key from the returned response. This is useful for when you are passing in a single tool and just want it’s arguments.

In [5]:
import { JsonOutputKeyToolsParser } from "langchain/output_parsers";

const outputParser = new JsonOutputKeyToolsParser({ keyName: "joke" });

In [17]:
const chain = prompt.pipe(llmWithTools).pipe(outputParser);

In [18]:
JSON.stringify(await chain.invoke({ topic: "Large Language Models" }), null, 2);

"[\n" +
  "  {\n" +
  '    "setup": "Why did the large language model go on a diet?",\n' +
  `    "punchline": "It couldn't s`... 38 more characters

> #### Inspect the [LangSmith trace](https://smith.langchain.com/public/698f4395-37dd-4c47-9dbe-31b0f9414987/r) with the `JsonOutputKeyToolsParser` output parser

Some LLMs have support for calling multiple tools in a single response. Because of this, the result of invoking `JsonOutputKeyToolsParser` is always an array. If you would only like a single result to be returned, you can specify `returnSingle` in the constructor.

In [38]:
const outputParserAll = new JsonOutputKeyToolsParser({
  keyName: "joke",
});
const outputParserSingle = new JsonOutputKeyToolsParser({
  keyName: "joke",
  returnSingle: true,
});

In [39]:
const chain = prompt.pipe(llmWithTools);

In [40]:
const response = await chain.invoke({ topic: "Large Language Models" });

Without `returnSingle`:

In [41]:
JSON.stringify(await outputParserAll.invoke(response));

'[{"setup":"Why did the large language model bring a flashlight?","punchline":"Because it heard it sh'... 49 more characters

> #### See the [LangSmith trace](https://smith.langchain.com/public/4733255d-d413-42a2-9094-d4f59b1c552c/r) from this output parser.

With `returnSingle`:

In [42]:
JSON.stringify(await outputParserSingle.invoke(response));

'{"setup":"Why did the large language model bring a flashlight?","punchline":"Because it heard it sho'... 47 more characters

> #### See the [LangSmith trace](https://smith.langchain.com/public/6f6225c0-ad4c-40c7-81c1-4f26d0e4b085/r) from this output parser.